# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [84]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [85]:
csv_path = "../output_data/cities.csv"
df = pd.read_csv(csv_path)
df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dikson,73.5069,80.5464,40.33,92,100,4.34,RU,1629672590
1,scarborough,54.2797,-0.4044,60.62,92,85,1.01,GB,1629672482
2,richards bay,-28.7830,32.0377,62.96,75,100,2.17,ZA,1629672590
3,ormara,25.2088,64.6357,78.39,89,46,11.10,PK,1629672590
4,punta arenas,-53.1500,-70.9167,37.51,93,90,10.36,CL,1629672591
5,tasiilaq,65.6145,-37.6368,44.76,94,82,3.04,GL,1629672591
6,carutapera,-1.1950,-46.0200,77.43,84,20,7.45,BR,1629672591
7,nhulunbuy,-12.2333,136.7667,81.21,78,90,14.97,AU,1629672592
8,khatanga,71.9667,102.5000,46.67,85,100,10.63,RU,1629672592
9,san pedro,14.3500,121.0167,78.31,92,20,4.61,PH,1629672592


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [100]:
gmaps.configure(api_key = g_key)

coordinates = df[['Lat', 'Lng']]
humidity = df['Humidity']


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)

#fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [101]:
ideal_weather = df.loc[(df['Max Temp']< 80) & (df['Max Temp']> 70) & (df['Wind Speed']< 10) & (df['Cloudiness']<10)].dropna()
ideal_weather.head(30)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,komsomolskiy,40.4272,71.7189,75.92,41,6,9.22,UZ,1629672616
120,caravelas,-17.7125,-39.2481,70.41,86,6,5.64,BR,1629672625
125,cidreira,-30.1811,-50.2056,71.19,61,5,6.42,BR,1629672627
166,domoni,-12.2569,44.5319,74.64,78,0,4.85,KM,1629672638
170,saint anthony,45.0205,-93.2180,79.72,47,1,5.01,US,1629672639
172,marrakesh,31.6315,-8.0083,78.87,53,0,4.61,MA,1629672640
185,cassilandia,-19.1133,-51.7342,78.73,27,0,3.02,BR,1629672643
191,torres,-29.3353,-49.7269,78.82,62,0,6.73,BR,1629672645
199,abha,18.2164,42.5053,70.05,18,0,0.00,SA,1629672648
236,ponta do sol,32.6667,-17.1000,72.23,78,0,3.22,PT,1629672260


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [102]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ''

hotel_params = {
    "type": 'hotel',
    "radius": 5000,
    'key': g_key   
}
place_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for  index, row in ideal_weather.iterrows():
    hotel_params['location'] = f"{row['Lat']}, {row['Lng']}"
    
    results = requests.get(place_url, params=hotel_params).json()
    hotel_df.loc[index, 'Hotel Name']=results['results'][1]['name']
hotel_df= pd.merge(hotel_df, ideal_weather, left_index=True, right_index=True)
hotel_df

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,Mirage,komsomolskiy,40.4272,71.7189,75.92,41,6,9.22,UZ,1629672616
120,Conselho Tutelar,caravelas,-17.7125,-39.2481,70.41,86,6,5.64,BR,1629672625
125,Diego Imóveis,cidreira,-30.1811,-50.2056,71.19,61,5,6.42,BR,1629672627
166,Hospital De Domoni,domoni,-12.2569,44.5319,74.64,78,0,4.85,KM,1629672638
170,"Holiday Inn Express Roseville-St. Paul, an IHG...",saint anthony,45.0205,-93.2180,79.72,47,1,5.01,US,1629672639
172,Novotel Marrakech Hivernage,marrakesh,31.6315,-8.0083,78.87,53,0,4.61,MA,1629672640
185,Conselho Tutelar,cassilandia,-19.1133,-51.7342,78.73,27,0,3.02,BR,1629672643
191,Farol Hotel,torres,-29.3353,-49.7269,78.82,62,0,6.73,BR,1629672645
199,Abha Hotel,abha,18.2164,42.5053,70.05,18,0,0.00,SA,1629672648
236,Hotel do Campo,ponta do sol,32.6667,-17.1000,72.23,78,0,3.22,PT,1629672260


In [103]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [104]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                            info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))